In [1]:
#Import Libraries
import numpy as np
import CoolProp.CoolProp as CP
import matplotlib.pyplot as plt

In [2]:
#Initial conditions

fluid_1 = "Air"
fluid_2 = "Water"
m_dot_air = 50 #kg/s
m_dot_water = 3 #kg/s
T0 = 298.15 #Kelvin
P0 = 1.01325e5 #Pa
P11 = 4e6 #Pa
P15 = 10e3 #Pa
T_max = T3 =  1100 #Kelvin | T @ Gas Turbine's inlet
h0 = CP.PropsSI("H", "P", P0, "T", T0, fluid_1)
s0 = CP.PropsSI("S", "P", P0, "T", T0, fluid_1)

#Fuel specification

#Fuel: Methane (CH4)
LHV = 50e6

#Number of flows in system

m = 17 

#Equipment specification

#Compressor

rp_comp = 5
eta_isen_comp = 0.8

#Combustion chamber

dp_cc = 0.01
eta_th_cc = 0.98

#Gas turbine

rp_gt = 4.5
eta_isen_gt = 0.89

#Heat exchanger 1

epsilon_HX1 = 0.95
eta_th_HX1 = 1
dp_HX1 = 0.02

#Heat exchanger 2

eta_th_HX2 = 1
dp_HX2 = 0.03

#Steam turbine

eta_isen_st = 0.95

#Condenser

eta_th_cond = 1
dp_cond = 0.02

#Pump

eta_isen_pump = 0.98

In [3]:
#Pressure matrix

P=np.zeros(m)

P[0] = P0
P[1] = P[0]*rp_comp
P[2] = P[1]*(1-dp_HX1)
P[3] = P[2]*(1-dp_cc)
P[7] = P[3]/rp_gt
P[8] = P[7]*(1-dp_HX2)
P[9] = P[8]*(1-dp_HX1)
P[11] = P11
P[10] = P[11]*(1+dp_HX1)
P[15] = P15
P[13] = P[15]/(1-dp_cond)

for i in range(m):
    if i in (4,5,6,12,14,16): 
        continue
    print(f"p of flow {i}: {P[i] / 1e3:.3f} KPa")

p of flow 0: 101.325 KPa
p of flow 1: 506.625 KPa
p of flow 2: 496.493 KPa
p of flow 3: 491.528 KPa
p of flow 7: 109.228 KPa
p of flow 8: 105.951 KPa
p of flow 9: 103.832 KPa
p of flow 10: 4080.000 KPa
p of flow 11: 4000.000 KPa
p of flow 13: 10.204 KPa
p of flow 15: 10.000 KPa


In [4]:
s1 = s0
h1s = CP.PropsSI("H", "P", P[1], "S", s1, fluid_1)

h3 = CP.PropsSI("H", "P", P[3], "T", T3, fluid_1)
s3 = CP.PropsSI("S", "P", P[3], "T", T3, fluid_1)

s7 = s3
h7s = CP.PropsSI("H", "P", P[7], "S", s7, fluid_1)

h11 = CP.PropsSI("H", "P", P[11], "Q", 1, fluid_2)
s11 = CP.PropsSI("S", "P", P[11], "Q", 1, fluid_2)
s13 = s11
h13s = CP.PropsSI("H", "P", P[13], "S", s13, fluid_2)

h15 = CP.PropsSI("H", "P", P[15], "Q", 0, fluid_2)
s15 = CP.PropsSI("S", "P", P[15], "Q", 0, fluid_2)
s10 = s15
h10s = CP.PropsSI("H", "P", P[10], "S", s10, fluid_2)

#Energy balance

A=np.zeros((m,m))
X=np.zeros(m)      #X=[h0,h1,h2,h3,Q_cc,w_gt,w_comp,h7,h8,h9,h10,h11,w_st,h13,Q_cond,h15,w_pump]
B=np.zeros(m)

#Compressor
A[0,0] = m_dot_air
A[0,1] = -m_dot_air
A[0,6] = 1
B[0] = 0

#HX1
A[1,1] = m_dot_air
A[1,2] = -m_dot_air
A[1,8] = m_dot_air
A[1,9] = -m_dot_air
B[1] = 0

#CC
A[2,2] = m_dot_air
A[2,3] = -m_dot_air
A[2,4] = 1
B[2] = 0

#GT
A[3,3] = m_dot_air
A[3,5] = -1
A[3,6] = -1
A[3,7] = -m_dot_air
B[3] = 0

#HX2
A[4,7] = m_dot_air
A[4,8] = -m_dot_air
A[4,10] = m_dot_water
A[4,11] = -m_dot_water
B[4] = 0

#ST
A[5,11] = m_dot_water
A[5,12] = -1
A[5,13] = -m_dot_water
B[5] = 0

#Condenser
A[6,13] = m_dot_water
A[6,14] = -1
A[6,15] = -m_dot_water
B[6] = 0

#Pump
A[7,10] = -m_dot_water
A[7,15] = m_dot_water
A[7,16] = 1
B[7] = 0

#Isentropic efficiency of Compressor
A[8,0] = 1-eta_isen_comp
A[8,1] = eta_isen_comp
B[8] = h1s

#Isentropic efficiency of Gas turbine
A[9,3] = 1-(1/eta_isen_gt)
A[9,7] = 1/eta_isen_gt
B[9] = h7s

#Isentropic efficiency of Steam turbine
A[10,11] = 1-(1/eta_isen_st)
A[10,13] = 1/eta_isen_st
B[10] = h13s

#Isentropic efficiency of Pump
A[11,15] = 1-eta_isen_pump
A[11,10] = eta_isen_pump
B[11] = h10s

#Effectiveness of HX1
A[12,1] = 1-epsilon_HX1
A[12,2] = -1
A[12,8] = epsilon_HX1
B[12] = 0

#known variables
#based on T0 and P0
A[13,0] = 1
B[13] = h0

#based on T3 and P3
A[14,3] = 1
B[14] = h3

#based on P15 and X
A[15,15] = 1
B[15] = h15

#based on P11 and X
A[16,11] = 1
B[16] = h11


X = np.linalg.solve(A, B)
for i in range(m):
    print(f"h of flow {i}: {X[i] / 1e3:.3f} kJ/kg")

h of flow 0: 424.436 kJ/kg
h of flow 1: 642.959 kJ/kg
h of flow 2: 772.762 kJ/kg
h of flow 3: 1287.782 kJ/kg
h of flow 4: 25750.970 kJ/kg
h of flow 5: 6668.762 kJ/kg
h of flow 6: 10926.153 kJ/kg
h of flow 7: 935.883 kJ/kg
h of flow 8: 779.594 kJ/kg
h of flow 9: 649.791 kJ/kg
h of flow 10: 195.998 kJ/kg
h of flow 11: 2800.821 kJ/kg
h of flow 12: 2502.321 kJ/kg
h of flow 13: 1966.714 kJ/kg
h of flow 14: 5324.723 kJ/kg
h of flow 15: 191.806 kJ/kg
h of flow 16: 12.576 kJ/kg


In [5]:
#Enthalpy

def enthalpy():

    enthalpy = np.zeros(m) #J/kg
    for i in range (m):
        if i in (4,5,6,12,14,16): 
            continue
        print(f"h of flow {i}: {X[i] / 1e3:.3f} kJ/kg")

enthalpy_results = enthalpy()
print(enthalpy_results)

h of flow 0: 424.436 kJ/kg
h of flow 1: 642.959 kJ/kg
h of flow 2: 772.762 kJ/kg
h of flow 3: 1287.782 kJ/kg
h of flow 7: 935.883 kJ/kg
h of flow 8: 779.594 kJ/kg
h of flow 9: 649.791 kJ/kg
h of flow 10: 195.998 kJ/kg
h of flow 11: 2800.821 kJ/kg
h of flow 13: 1966.714 kJ/kg
h of flow 15: 191.806 kJ/kg
None


In [5]:
def turbines_works():
    
    GT_work = X[5]
    ST_work = X[12]

    return [GT_work, ST_work]

turbines_works = turbines_works()
print("GT work is ", turbines_works[0]/1e6, "MW")
print("ST work is ", turbines_works[1]/1e6, "MW")


GT work is  6.66876184157082 MW
ST work is  2.5023208986880956 MW


In [7]:
def comp_pump_works():
    
    comp_work = X[6]
    pump_work = X[16]

    return [comp_work, pump_work]

comp_pump_works = comp_pump_works()
print("comp work is ", comp_pump_works[0]/1e6, "MW")
print("pump work is ", comp_pump_works[1]/1e3, "KW")

comp work is  10.926153340750977 MW
pump work is  12.576252373840246 KW


In [8]:
#Temperature

T = np.zeros(m) #Kelvin
for i in range (0,10):
    if i not in (4,5,6):
        T[i]= CP.PropsSI("T", "P", P[i], "H", X[i], fluid_1)
for i in range (10,16):
    if i not in (12,14,16):
        T[i]= CP.PropsSI("T", "P", P[i], "H", X[i], fluid_2)

print(T)

[ 298.15        513.33111513  637.28221504 1100.            0.
    0.            0.          788.58548371  643.71703279  519.76848057
  319.11106767  523.50404529    0.          319.35204977    0.
  318.95632892    0.        ]


In [9]:
#Entropy

def entropy():

    s = np.zeros(m)
    for i in range(m):
        if i in (4,5,6,12,14,16): 
            continue 
        elif i in (0,1,2,3,7,8,9):
            s[i] = CP.PropsSI("S", "P", P[i], "H", X[i], fluid_1)
        elif i in (10,11,13,15):
            s[i] = CP.PropsSI("S", "P", P[i], "H", X[i], fluid_2)
        else:
            print("Something is wrong!!!")
    return s
    
entropies = entropy()
for i in range(m):
    if i in (4,5,6,12,14,16): 
        continue
    print(f"s of flow {i}: {entropies[i] / 1e3:.3f} kJ/kg.K")


s of flow 0: 3.880 kJ/kg.K
s of flow 1: 3.969 kJ/kg.K
s of flow 2: 4.202 kJ/kg.K
s of flow 3: 4.810 kJ/kg.K
s of flow 7: 4.866 kJ/kg.K
s of flow 8: 4.656 kJ/kg.K
s of flow 9: 4.438 kJ/kg.K
s of flow 10: 0.649 kJ/kg.K
s of flow 11: 6.070 kJ/kg.K
s of flow 13: 6.207 kJ/kg.K
s of flow 15: 0.649 kJ/kg.K


In [10]:
#m_dot_fuel

def m_dot_fuel():

    Q_CC = X[4]
    m_dot_fuel = Q_CC / (eta_th_cc * LHV) 

    return m_dot_fuel

m_dot_fuel_res = m_dot_fuel()
print("M_dot_fuel is: ",m_dot_fuel_res, "Kg/s")

M_dot_fuel is:  0.5255300056257989 Kg/s


In [11]:
#Exergy

def exergy():
        s = entropy()
        EX = np.zeros(m)
        for i in range(0,10):
            if i in (4,5,6):
                EX[i] = 0
            else:
                EX[i] = m_dot_air * ((X[i]-X[0]) - T0*(s[i]-s[0]))
            
        for i in range(10,16):
            if i in (12,14,16): 
                EX[i] = 0
            else:
                EX[i] = m_dot_water * ((X[i]-X[0]) - T0*(s[i]-s[0]))

            
                  
            


        EX[5] = X[5]
        EX[6] = X[6]
        EX[12] = X[12]
        EX[16] = X[16]

        ex_ch_fuel = 51.94e6 #J/Kg #Methane

        EX[4] = m_dot_fuel() * ex_ch_fuel

        T13 = CP.PropsSI("T", "P", P[13], "H", X[13], fluid_2)
        T15 = CP.PropsSI("T", "P", P[15], "H", X[15], fluid_2)
        T_avg_cond = (T13 + T15)/2

        EX[14] = (1 - T0/T_avg_cond) * X[14]

        return EX

exergy_results = exergy()
print (exergy_results)

exergy_results = exergy()
for i in range(m):
    
    print(f"Ex of flow {i}: {exergy_results[i] / 1e6:.3f} MW")

[0.00000000e+00 9.60149579e+06 1.26300051e+07 2.93175835e+07
 2.72960285e+07 6.66876184e+06 1.09261533e+07 1.08790000e+07
 6.19686991e+06 2.95760168e+06 2.20468147e+06 5.17112901e+06
 2.50232090e+06 2.54585076e+06 3.50430912e+05 2.19234023e+06
 1.25762524e+04]
Ex of flow 0: 0.000 MW
Ex of flow 1: 9.601 MW
Ex of flow 2: 12.630 MW
Ex of flow 3: 29.318 MW
Ex of flow 4: 27.296 MW
Ex of flow 5: 6.669 MW
Ex of flow 6: 10.926 MW
Ex of flow 7: 10.879 MW
Ex of flow 8: 6.197 MW
Ex of flow 9: 2.958 MW
Ex of flow 10: 2.205 MW
Ex of flow 11: 5.171 MW
Ex of flow 12: 2.502 MW
Ex of flow 13: 2.546 MW
Ex of flow 14: 0.350 MW
Ex of flow 15: 2.192 MW
Ex of flow 16: 0.013 MW


In [12]:
#eta_th_plant

def eta_th_plant():

    W_net = X[5] + X[12] - X[16]

    eta_th_plant = (W_net / (X[4])) * 100

    return eta_th_plant

eta_th_plant_result = eta_th_plant()
print(eta_th_plant_result)

35.56567535064994


In [13]:
#Exergy Destruction

def exergy_des():

    EX = exergy()
    EX_Des = np.zeros(8)
    
    #Comp
    EX_Des[0] = EX[0] - EX[1] + EX[6]

    #HX1
    EX_Des[1] = EX[1] + EX[8] - EX[2] - EX[9]

    #CC
    EX_Des[2] = EX[2] - EX[3] + EX[4]

    #GT
    EX_Des[3] = EX[3] - EX[7] - EX[5]

    #HX2
    EX_Des[4] = EX[7] - EX[8] + EX[10] - EX[11]

    #ST
    EX_Des[5] = EX[11] - EX[13] - EX[12]

    #Cond
    EX_Des[6] = EX[13] - EX[15] - EX[14]

    #Pump
    EX_Des[7] = EX[15] - EX[10] + EX[16]

    return EX_Des


EX_Destruction_results = exergy_des()
for i in range(8):
    
    print(f"Ex_Destruction of equipment {i}: {EX_Destruction_results[i] / 1e6:.5f} MW")


Ex_Destruction of equipment 0: 1.32466 MW
Ex_Destruction of equipment 1: 0.21076 MW
Ex_Destruction of equipment 2: 10.60845 MW
Ex_Destruction of equipment 3: 11.76982 MW
Ex_Destruction of equipment 4: 1.71568 MW
Ex_Destruction of equipment 5: 0.12296 MW
Ex_Destruction of equipment 6: 0.00308 MW
Ex_Destruction of equipment 7: 0.00024 MW


In [14]:
#Exergy_Efficiency

def exergy_efficiency():

    EX = exergy()
    EX_eff = np.zeros(8)
    
    #Comp
    EX_eff[0] = ((EX[1]-EX[0]) / EX[6]) * 100

    #HX1
    EX_eff[1] = (EX[2]-EX[1]) / (EX[8]-EX[9]) * 100

    #CC
    EX_eff[2] = ((EX[3] - EX[2]) / EX[4]) * 100

    #GT
    EX_eff[3] = (EX[5] / (EX[3] - EX[7])) * 100

    #HX2
    EX_eff[4] = ((EX[11] - EX[10]) / (EX[7] - EX[8])) * 100

    #ST
    EX_eff[5] = (EX[12] / (EX[11] - EX[13])) * 100

    #Cond
    # EX_eff[6] = (EX[14] / (EX[13] - EX[15])) * 100
    EX_eff[6] = (EX[15] / EX[13]) * 100

    #Pump
    EX_eff[7] = ((EX[10] - EX[15]) / EX[16]) * 100

    return EX_eff


EX_efficiency_results = exergy_efficiency()
for i in range(8):
    
    print(f"Ex_efficiency of equipment {i}: {EX_efficiency_results[i]:.2f} %")
    

Ex_efficiency of equipment 0: 87.88 %
Ex_efficiency of equipment 1: 93.49 %
Ex_efficiency of equipment 2: 61.14 %
Ex_efficiency of equipment 3: 36.17 %
Ex_efficiency of equipment 4: 63.36 %
Ex_efficiency of equipment 5: 95.32 %
Ex_efficiency of equipment 6: 86.11 %
Ex_efficiency of equipment 7: 98.13 %


In [15]:
#Plant_Exergy_Destruction

def plant_EX_Des():

    EX = exergy()
    EX_Des = exergy_des()

    Sum_EX = sum(EX_Des)

    return Sum_EX

Plant_Exergy_Destruction = plant_EX_Des()
print(f"Ex_Destruction of the plant: {Plant_Exergy_Destruction / 1e6:.5f} MW")


Ex_Destruction of the plant: 25.75564 MW


In [16]:
#Plant_Exergy_Efficiency

def plant_EX_eff():

    EX = exergy()
    
    return ((EX[5] + EX[12] - EX[16]) / EX[4]) * 100

Plant_Exergy_Efficiency = plant_EX_eff()
print(f"Ex_efficiency of the plant: {Plant_Exergy_Efficiency:.5f} %")


Ex_efficiency of the plant: 33.55252 %
